# Direct location with DEM
### PyRugged API - tutorial 2

The aim of this tutorial is to compute a direct location grid by intersection of the line of sight with a DEM (Digital Elevation Model), using Duvenhage’s algorithm. This algorithm is the most performant one in `PyRugged`.

---

__Module imports__

In [1]:
import orekit
import numpy as np 

import os 
import sys

from java.io import File

from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.data import DataContext
from org.orekit.data import DirectoryCrawler
from org.orekit.frames import Frame
from org.orekit.frames import FramesFactory
from org.orekit.frames import Transform
from org.orekit.time import AbsoluteDate
from org.orekit.time import TimeScalesFactory
from org.orekit.utils import AngularDerivativesFilter
from org.orekit.utils import CartesianDerivativesFilter
from org.orekit.utils import IERSConventions
from org.orekit.utils import TimeStampedAngularCoordinates
from org.orekit.utils import TimeStampedPVCoordinates

from pyrugged.configuration.init_orekit import init_orekit
from pyrugged.intersection.algorithm_id import AlgorithmId
from pyrugged.intersection.intersection_algorithm import create_intersection_algorithm
from pyrugged.bodies.body_rotating_frame_id import BodyRotatingFrameId
from pyrugged.bodies.ellipsoid_id import EllipsoidId
from pyrugged.model.inertial_frame_id import InertialFrameId
from pyrugged.model.pyrugged_builder import PyRuggedBuilder
from pyrugged.line_sensor.line_sensor import LineSensor
from pyrugged.line_sensor.linear_line_datation import LinearLineDatation
from pyrugged.los.fixed_rotation import FixedRotation
from pyrugged.los.los_builder import LOSBuilder
from pyrugged.bodies.geodetic_point import GeodeticPoint
from pyrugged.location.optical import OpticalLocation


# add helpers modules from tests
import os
import sys
module_path = os.path.join(os.getcwd(), "./../tests/")
if module_path not in sys.path:
    sys.path.append(module_path)

import helpers
from raster.volcanic_cone_elevation_updater import VolcanicConeElevationUpdater 


### Initialization 

initialize JVM or GraalVM subtrat VM, and initialize Orekit, using internal orekit data 

In [2]:
init_orekit()

## 1 - Sensor's definition

### Line of sight
The raw viewing direction of pixel i with respect to the instrument is defined by the vector:

In [3]:
raw_dirs = []
for i in range(2000):
    raw_dirs.append(
        Vector3D(0.0, i * float(np.radians(20.0)) / 2000.0, 1.0)
    )

The instrument is oriented 10° off nadir around the X-axis. 
We need to rotate the viewing direction to obtain the line of sight in the satellite frame.

In [4]:
los_builder = LOSBuilder(raw_dirs)
los_builder.add_ti_los_transform(
    FixedRotation(
        "10-degrees-rotation",
        Vector3D.PLUS_I,
        np.radians(10.)
    )
)

line_of_sight = los_builder.build()

### Datation model
We use Orekit for handling time and dates, and PyRugged for defining the datation model:

In [5]:
gps = TimeScalesFactory.getGPS()
abs_date = AbsoluteDate("2009-12-11T16:59:30.0", gps)
line_datation = LinearLineDatation(
    abs_date, 1.0, 20
)

### Line sensor
With the LOS and the datation now defined, we can initialize a line sensor object in PyRugged:

In [6]:
line_sensor = LineSensor(
    "my-sensor",
    line_datation,
    Vector3D.ZERO,
    line_of_sight,
)

## 2 - Satellite position, velocity and attitude

### Reference frames
In our application, we simply need to know the name of the frames we are working with. Positions and
velocities are given in the ITRF terrestrial frame, while the quaternions are given in EME2000
inertial frame.

In [7]:
eme_2000 = FramesFactory.getEME2000()

# We don't want to compute tiny tidal effects at millimeter level
simple_eop = True

itrf = FramesFactory.getITRF(
    IERSConventions.IERS_2010, simple_eop
)

### Satellite attitude

In [8]:
satellite_q_list = []

helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:58:42.592937", -0.340236, 0.333952, -0.844012, -0.245684)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:06.592937", -0.354773, 0.329336, -0.837871, -0.252281)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:30.592937", -0.369237, 0.324612, -0.831445, -0.258824)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:54.592937", -0.3836, 0.319792, -0.824743, -0.265299)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:00:18.592937", -0.397834, 0.314883, -0.817777, -0.271695)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:00:42.592937", -0.411912, 0.309895, -0.810561, -0.278001)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:06.592937", -0.42581, 0.304838, -0.803111, -0.284206)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:30.592937", -0.439505, 0.299722, -0.795442, -0.290301)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:54.592937", -0.452976, 0.294556, -0.787571, -0.296279)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:02:18.592937", -0.466207, 0.28935, -0.779516, -0.302131)


### Positions and velocities

In [9]:
satellite_pv_list = []

helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:58:42.592937", -726361.466, -5411878.485, 4637549.599, -2463.635, -4447.634, -5576.736)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:04.192937", -779538.267, -5506500.533, 4515934.894, -2459.848, -4312.676, -5683.906)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:25.792937", -832615.368, -5598184.195, 4392036.13, -2454.395, -4175.564, -5788.201)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:47.392937", -885556.748, -5686883.696, 4265915.971, -2447.273, -4036.368, -5889.568)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:08.992937", -938326.32, -5772554.875, 4137638.207, -2438.478, -3895.166, -5987.957)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:30.592937", -990887.942, -5855155.21, 4007267.717, -2428.011, -3752.034, -6083.317)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:52.192937", -1043205.448, -5934643.836, 3874870.441, -2415.868, -3607.05, -6175.6)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:13.792937", -1095242.669, -6010981.571, 3740513.34, -2402.051, -3460.291, -6264.76)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:35.392937", -1146963.457, -6084130.93, 3604264.372, -2386.561, -3311.835, -6350.751)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:56.992937", -1198331.706, -6154056.146, 3466192.446, -2369.401, -3161.764, -6433.531)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:02:18.592937", -1249311.381, -6220723.191, 3326367.397, -2350.574, -3010.159, -6513.056)

## 3 - PyRugged initialization

In [10]:
builder = PyRuggedBuilder()


builder.set_ellipsoid(
    new_ellipsoid = None,
    ellipsoid_id = EllipsoidId.WGS84,
    body_rotating_frame_id = BodyRotatingFrameId.ITRF,
)

builder.set_time_span(
    abs_date,
    abs_date.shiftedBy(60.0),
    0.01,
    5 / line_sensor.get_rate(0)
)

builder.set_trajectory(
    satellite_pv_list,
    4, CartesianDerivativesFilter.USE_P,
    satellite_q_list, 
    4, AngularDerivativesFilter.USE_R,
    inertial_frame_id=InertialFrameId.EME2000,
)

builder.add_line_sensor(line_sensor)

rugged = builder.build()

## 4 - intersection algorithm initialization

- The initialization step differs slightly from the first tutorial Direct location, as we need to pass the information about our tile updater.

- We use Duvenhage's algorithm which is the main intersection algorithm .

__Feeding PyRugged with DEM tiles__

`PyRugged` does not parse DEM files but takes buffers of elevation data as input. It is up to the calling application to read the DEM and load the data into buffers. `PyRugged` provides a tile mecanism with cache for large DEMs allowing the user to load one tile at a time. This is in line with the format of world coverage DEMs such as SRTM. `PyRugged` offers an interface for updating the DEM tiles in cache with a callback function triggered everytime a coordinate falls outside the current region.

The calling application must implement the callback function for loading the tiles. We recommend to use GDAL (http://www.gdal.org/) to parse the DEM files as it handles most formats (including geoTIFF for Aster DEM or DTED for SRTM). `PyRugged` does not include the parsing of the DEM, by design, and yet we could have used GDAL. We want `PyRugged` to remain a low-level library that does not pull too many third-party libraries.

__PyRugged tile updaters for test cases__

In this tutorial, we will not include real DEM data. Instead we are going to create a fake DEM representing a volcano in a form of a perfect cone, similar to the Mayon volcano in the Philippines, except that we will locate it somewhere just below our satellite. This example is already part of `PyRugged` tests cases, the source code is available in the package `pyrugged/tests/raster`, file `volcanic_cone_elevation_updater.py`.

The class VolcanicConeElevationUpdater inherits from `TileUpdater` which has the abstract method `update_tile`. The method is in charge of loading a tile. The extent of the tile must be such that it covers at least the ground point with coordinates (latitude, longitude) which are passed as arguments to the method. 

__Important notes on DEM tiles__

- Ground point elevation are obtained by bilinear interpolation between 4 neighbouring cells. There is no specific algorithm for border management. As a consequence, a point falling on the border of the tile is considered outside. DEM tiles must be overlapping by at least one line/column in all directions.


- In `PyRugged` terminology, the minimum latitude and longitude correspond to the centre of the farthest Southwest cell of the DEM. Be careful if using GDAL to pass the correct information as there is half a pixel shift with respect to the lower left corner coordinates in gdalinfo.


In [11]:
updater = VolcanicConeElevationUpdater(
        GeodeticPoint(
            float(np.radians(37.58)), 
            float(np.radians(-96.95)),
            2463.0,
        ),
        float(np.radians(30.0)), 16.0,
        float(np.radians(1.0)), 1201,
    )

intersection_algorithm = create_intersection_algorithm(AlgorithmId.DUVENHAGE, updater, 8)

location = OpticalLocation(rugged, intersection_algorithm)

## 5 - Direct location of a single sensor pixel

In [12]:
position = line_sensor.position
first_line_date = line_sensor.get_date(0)
los = line_sensor.get_los(first_line_date, 0)

up_left_point = location.direct_location_of_los(
    first_line_date,
    position, 
    los,
)

print("longitude {:.8f}° latitude {:.8f}° altitude {:.2f}m".format(np.degrees(up_left_point[0]), np.degrees(up_left_point[1]),up_left_point[2]))

longitude -96.95384304° latitude 37.58546815° altitude 2090.29m
